In [1]:
!conda install -y -c conda-forge jupyterlab_widgets
!conda install -y -c conda-forge ipywidgets

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - jupyterlab_widgets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    jupyterlab_widgets-1.0.2   |     pyhd8ed1ab_0         130 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         130 KB

The following NEW packages will be INSTALLED:

  jupyterlab_widgets conda-forge/noarch::jupyterlab_widgets-1.0.2-pyhd8ed1ab_0



jupyterlab_widgets-1 | 130 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - ipywidgets


The following packages will be downloaded:

    package                    |            bui

In [2]:
import os
import torch
import torchvision
import pandas as pd
from google.cloud import storage

In [3]:
# create a storage bucket
client = storage.Client()
bucket = client.bucket('cv-with-jake-fashionmnist')
bucket.create(location='us-central1')

In [4]:
# download data
fashion_train = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True)
fashion_test = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [5]:
if not os.path.exists('./data/FashionMNIST/train'):
    os.mkdir('./data/FashionMNIST/train')

if not os.path.exists('./data/FashionMNIST/test'):
    os.mkdir('./data/FashionMNIST/test')

In [6]:
automl_label_df = pd.DataFrame(columns=["ML_USE", "GCS_FILE_PATH", "LABEL"])
train_df = pd.DataFrame(columns=["LOCAL_FILE_PATH", "LABEL", "GCS_FILE_PATH"])
test_df = pd.DataFrame(columns=["LOCAL_FILE_PATH", "LABEL", "GCS_FILE_PATH"])

In [7]:
GCS_TRAIN_PATH = 'gs://cv-with-jake-fashionmnist/train'
BASE_TRAIN_PATH = './data/FashionMNIST/train'
for i in range(0, len(fashion_train)):
    photo_name = f'img_{i}.jpg'
    item = fashion_train[i]
    photo = item[0]
    label = item[1]
    automl_label_df = automl_label_df.append({'ML_USE': 'UNASSIGNED', 'GCS_FILE_PATH': f'{GCS_TRAIN_PATH}/{photo_name}', 'LABEL': label}, ignore_index=True)
    train_df = train_df.append({'LOCAL_FILE_PATH': f'{BASE_TRAIN_PATH}/{photo_name}', 'LABEL': label, 'GCS_FILE_PATH': f'{GCS_TRAIN_PATH}/{photo_name}'}, ignore_index=True)
    photo.save(f'{BASE_TRAIN_PATH}/{photo_name}')

In [8]:
train_df.to_csv('./train_files.csv', index=False)

In [9]:
GCS_TEST_PATH = 'gs://cv-with-jake-fashionmnist/test'
BASE_TEST_PATH = './data/FashionMNIST/test'
for i in range(0, len(fashion_test)):
    photo_name = f'img_{i}.jpg'
    item = fashion_test[i]
    photo = item[0]
    label = item[1]
    automl_label_df = automl_label_df.append({'ML_USE': 'TEST', 'GCS_FILE_PATH': f'{GCS_TEST_PATH}/{photo_name}', 'LABEL': label}, ignore_index=True)
    test_df = test_df.append({'LOCAL_FILE_PATH': f'{BASE_TEST_PATH}/{photo_name}', 'LABEL': label, 'GCS_FILE_PATH': f'{GCS_TEST_PATH}/{photo_name}'}, ignore_index=True)
    photo.save(f'{BASE_TEST_PATH}/{photo_name}')

In [10]:
test_df.to_csv('./test_files.csv', index=False)

In [11]:
!gsutil -m -q cp -r ./data/FashionMNIST/train gs://cv-with-jake-fashionmnist
!gsutil -m -q cp -r ./data/FashionMNIST/test gs://cv-with-jake-fashionmnist

In [12]:
automl_label_df.to_csv('./automl_file_list.csv', index=False, header=False)

In [13]:
!gsutil cp ./automl_file_list.csv gs://cv-with-jake-fashionmnist/automl_file_list.csv

Copying file://./automl_file_list.csv [Content-Type=text/csv]...
/ [1 files][  4.2 MiB/  4.2 MiB]                                                
Operation completed over 1 objects/4.2 MiB.                                      


In [14]:
test_files_df = test_df.copy()
test_files_df['content'] = test_files_df['GCS_FILE_PATH']
test_files_df['mimeType'] = 'image/jpeg'
test_files_df = test_files_df[['content', 'mimeType']]
test_files_df.to_json('./automl_test_list.jsonl', lines=True, orient='records')

In [15]:
!gsutil cp ./automl_test_list.jsonl gs://cv-with-jake-fashionmnist/automl_test_list.jsonl

Copying file://./automl_test_list.jsonl [Content-Type=application/octet-stream]...
/ [1 files][897.4 KiB/897.4 KiB]                                                
Operation completed over 1 objects/897.4 KiB.                                    
